# 1. tokenizer 加载预训练字典和分词方法

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',
    cache_dir=None,
    force_download=False,
)

sents = [
    '选择珠江花园的原因就是方便。',
    '笔记本的键盘确实爽。',
    '房间太小。其他的都一般。',
    '今天才知道这书还有第6卷,真有点郁闷.',
    '机器背面似乎被撕了张什么标签，残胶还在。',
]

tokenizer, sents

(PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 ['选择珠江花园的原因就是方便。',
  '笔记本的键盘确实爽。',
  '房间太小。其他的都一般。',
  '今天才知道这书还有第6卷,真有点郁闷.',
  '机器背面似乎被撕了张什么标签，残胶还在。'])

## 2.1 普通句子编码

In [24]:
out = tokenizer.encode(
    text=sents[0],
    text_pair=sents[1],
    # 大于maxlen，是否截断
    truncation=True,

    # 补齐pad到max_len
    padding='max_length',
    add_special_tokens=True,
    max_length=30,
    # pt, tf, np, ajax, 默认list
    return_tensors=None
)

print(out)
print(len(out))
tokenizer.decode(out)

[101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 511, 102, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0]
30


'[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 。 [SEP] 笔 记 本 的 键 盘 确 实 爽 。 [SEP] [PAD] [PAD] [PAD]'

### 2.2 增强型编码

In [29]:
out = tokenizer.batch_encode_plus(
    batch_text_or_text_pairs=[(sents[0], sents[1]), (sents[2], sents[3]), ],

    # 此处参数同上
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    max_length=30,
    return_tensors=None,

    # 返回新增的结果
    return_token_type_ids=True,
    return_length=True,
    return_attention_mask=True,
    return_special_tokens_mask=True,

    #返回offset_mapping 标识每个词的起止位置,这个参数只能BertTokenizerFast使用
    #return_offsets_mapping=True,
)
print(out)
#input_ids: 就是编码后的词
#token_type_ids: 第一个句子和特殊符号的位置是0,第二个句子的位置是1
#special_tokens_mask: 特殊符号的位置是1,其他位置是0
#attention_mask: pad的位置是0,其他位置是1
#length: 返回句子长度

tokenizer.decode(out['input_ids'][0])
tokenizer.decode(out['input_ids'][1])

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': [[101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 511, 102, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0], [101, 2791, 7313, 1922, 2207, 511, 1071, 800, 4638, 6963, 671, 5663, 511, 102, 791, 1921, 2798, 4761, 6887, 6821, 741, 6820, 3300, 5018, 127, 1318, 117, 4696, 3300, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'special_tokens_mask': [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], 'length': [27, 30], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


'[CLS] 房 间 太 小 。 其 他 的 都 一 般 。 [SEP] 今 天 才 知 道 这 书 还 有 第 6 卷, 真 有 [SEP]'

## 3. vocab的操作

In [37]:
zidian = tokenizer.get_vocab()

print(type(zidian))
print(len(zidian))
print('月光' in zidian)

<class 'dict'>
21128
False


## 3.2 添加token到字典

In [46]:
tokenizer.add_tokens(new_tokens=['月光', '希望'])

tokenizer.add_special_tokens({'eos_token':'[EOS]'})

zidian = tokenizer.get_vocab()

print(type(zidian))
print(len(zidian))
print('月光' in zidian)

<class 'dict'>
21131
True


## 3.3 编码新词

In [49]:
out = tokenizer.encode(
    text='月光新希望[EOS]',

    truncation=True,
    padding='max_length',
    max_length=15,
    return_tensors=None
)
print(out)

print(tokenizer.decode(out))

[101, 21128, 3173, 21129, 21130, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] 月光 新 希望 [EOS] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
